In [2]:
import sys
sys.path.insert(0, '../scripts')
import pipeline
import pandas as pd

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# ML Analysis

We start by exporting the data:

In [4]:
file = '../data/work/block-groups_2012-2016_with-acs.csv'
df = pipeline.read(file)

Defining the train/test sets:

In [8]:
years = [2013, 2014, 2015]
dic_sets = {}
for year in years:
    next_year = year + 1
    dic_sets['train_in_' + str(year)] = (df[df['year']==year], df[df['year']==next_year])

Defining the feature columns:

Defining the label: